In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import spectools.lpsd as lpsd
from spectools import analyzer
import multiprocessing as mp
ctx = mp.get_context('fork')
pool = ctx.Pool()

%load_ext autoreload
%autoreload 2

In [ ]:
size = int(1e6)
rho = 0.5

# Input signal:
x = 1e-5*np.random.normal(size=size) + np.sum([ np.cos(np.arange(size)/(2**i)) for i in range(30)], axis=0) \
    + np.cumsum(np.cumsum(1e-5*np.random.normal(size=size)))

# Output signal with no input influence:
y_pure = 1e-3*np.random.normal(size=size)

# Output signal with input influence:
y = rho * x + y_pure

fig, ax = plt.subplots()
ax.plot(x, label='Input')
ax.plot(y, label='Output with input influence')
ax.plot(y_pure, label='Output without input influence')
ax.set_xlabel('Sample')
ax.set_ylabel('Signal')
ax.legend()
plt.show()

In [ ]:
obj1 = lpsd.ltf([x, y], fs=2, pool=pool)
obj2 = lpsd.ltf(y_pure, fs=2, pool=pool)

In [ ]:
fig, ax = plt.subplots(figsize=(5,4), dpi=150)
ax.loglog(obj1.f, obj1.Gxx, label=r'$G_{xx}$ (autospectrum of input)', ls='-')
ax.loglog(obj1.f, obj1.Gyy, label=r'$G_{yy}$ (autospectrum of output)', ls='--')
ax.loglog(obj1.f, obj1.GyyCx, label=r'$G_{yy:x}$ (coherent output spectrum)', ls=':')
ax.loglog(obj1.f, obj1.GyyRx, label=r'$G_{yy \cdot x}$ (conditioned/residual output spectrum)')
ax.loglog(obj1.f, obj1.GyySx, label=r'Output spectrum with input influence subtracted', ls='--')
ax.scatter(obj2.f, obj2.psd, label=r'Actual output spectrum with no input influence', c='gray', s=5)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel('Power spectral density')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1.), edgecolor='black', fancybox=True, shadow=True, framealpha=1)
ax.grid(which='both')
ax.set_xlim(obj2.f[0], obj2.f[-1])
# ax.set_ylim(1e-11,1e6)
plt.show()